# TESLA Stop Loss Strategy
In order to derermine the optimum stop loss position for my TSLA shares, I will calculate the following from the last two years of TSLA candle data:
* Largest daily fall as amount and percentage;
* Largest intra day swing;
* Largest overnight fall as amount and percentage; and
* Largest weekly fall as amount and percentage.

## Get the TSLA candle data
Last two years from yahoo finance. Make sure it is sorted by date.

In [2]:
from pandas_datareader import data as dr
import datetime as dt

 # Get 2 years data + 1 extra week which we will drop after adding previous close and previous week ago close
from_date = dt.datetime.now() - dt.timedelta(days=365*2+7)
tsla_candles = dr.DataReader('TSLA', data_source='yahoo', start=from_date)

# Sort
tsla_candles = tsla_candles.sort_index()

tsla_candles

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-03-23,88.400002,82.099998,86.720001,86.858002,82272500.0,86.858002
2020-03-24,102.737999,94.800003,95.459999,101.000000,114476000.0,101.000000
2020-03-25,111.400002,102.222000,109.050003,107.849998,106113500.0,107.849998
2020-03-26,112.000000,102.449997,109.477997,105.632004,86903500.0,105.632004
2020-03-27,105.160004,98.806000,101.000000,102.872002,71887000.0,102.872002
...,...,...,...,...,...,...
2022-03-24,1024.489990,988.799988,1009.729980,1013.919983,22973600.0,1013.919983
2022-03-25,1021.799988,997.320007,1008.000000,1010.640015,20642900.0,1010.640015
2022-03-28,1097.880005,1053.599976,1065.099976,1091.839966,34168700.0,1091.839966


## Add the calculations
Eight new columns:
* previous close
* week ago close
* open vs previous close
* open vs week ago close
* close vs open
* swing (high - low)
* open vs prev close %
* open vs week ago close %
* high vs low %
* swing %

In [3]:
# Copy data so that we don't alter retrieved data frame
data = tsla_candles.copy()

# Add previous close and week ago close
data['Prev Close'] = tsla_candles["Close"].shift(1)
data['Wk Ago Close'] = tsla_candles["Close"].shift(7)

# Remove first weeks rows, which will contain NaN values
data = data.iloc[7: , :].copy()

# Add the increases (open vs previous close; open vs week ago close; Close - Open; and Swing)
data['Open vs Prev Close'] = (data["Open"] - data["Prev Close"])
data['Open vs Wk Ago Close'] = (data["Open"] - data["Wk Ago Close"])
data['Close vs Open'] = (data["Close"] - data["Open"])
data['Swing'] = (data["High"] - data["Low"])

# Add the increases as a %
data['Open vs Prev Close %'] = (data["Open vs Prev Close"] / data["Prev Close"] * 100)
data['Open vs Wk Ago Close %'] = (data["Open vs Wk Ago Close"] / data["Wk Ago Close"] * 100)
data['Close vs Open %'] = (data["Close vs Open"] / data["Open"] * 100)
data['Swing %'] = (data['Swing'] / data['Low'] * 100)

data                                     

,High,Low,Open,Close,Volume,Adj Close,Prev Close,Wk Ago Close,Open vs Prev Close,Open vs Wk Ago Close,Close vs Open,Swing,Open vs Prev Close %,Open vs Wk Ago Close %,Close vs Open %,Swing %
Date,,,,,,,,,,,,,,,,
2020-04-01,102.790001,95.019997,100.800003,96.311996,66766000.0,96.311996,104.800003,86.858002,-4.000000,13.942001,-4.488007,7.770004,-3.816794,16.051488,-4.452387,8.177231
2020-04-02,98.851997,89.279999,96.206001,90.893997,99292000.0,90.893997,96.311996,101.000000,-0.105995,-4.793999,-5.312004,9.571999,-0.110054,-4.746533,-5.521489,10.721325
2020-04-03,103.098000,93.678001,101.900002,96.001999,112810500.0,96.001999,90.893997,107.849998,11.006004,-5.949997,-5.898003,9.419998,12.108615,-5.516919,-5.788030,10.055721
2020-04-06,104.199997,99.592003,102.239998,103.248001,74509000.0,103.248001,96.001999,105.632004,6.237999,-3.392006,1.008003,4.607994,6.497780,-3.211154,0.985919,4.626872
2020-04-07,113.000000,106.468002,109.000000,109.089996,89599000.0,109.089996,103.248001,102.872002,5.751999,6.127998,0.089996,6.531998,5.571051,5.956916,0.082565,6.135174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-24,1024.489990,988.799988,1009.729980,1013.919983,22973600.0,1013.919983,999.109985,801.890015,10.619995,207.839966,4.190002,35.690002,1.062946,25.918762,0.414963,3.609426
2022-03-25,1021.799988,997.320007,1008.000000,1010.640015,20642900.0,1010.640015,1013.919983,840.229980,-5.919983,167.770020,2.640015,24.479980,-0.583871,19.967155,0.261906,2.454576
2022-03-28,1097.880005,1053.599976,1065.099976,1091.839966,34168700.0,1091.839966,1010.640015,871.599976,54.459961,193.500000,26.739990,44.280029,5.388661,22.200551,2.510562,4.202736


## Output
Display the:
* Largest daily fall as amount and percentage;
* Largest overnight fall as amount and percentage;
* Largest weekly fall as amount and percentage; and
* Largest swing for a bearish day.

In [4]:
# Format the data so that date time is in dd MMM yyyy
fmt_data = data.copy()
fmt_data.index = fmt_data.index.strftime('%d-%b-%Y')

# Largest daily fall by value
ldf_amt = fmt_data[fmt_data['Close vs Open'] == fmt_data['Close vs Open'].min()]
display(ldf_amt[['Open', 'Close', 'Close vs Open', 'Close vs Open %']].\
        style.set_caption("Largest Daily Fall by Value").format(precision=2))

# Largest daily fall by %
ldf_amt_pct = fmt_data[fmt_data['Close vs Open %'] == fmt_data['Close vs Open %'].min()]
display(ldf_amt_pct[['Open', 'Close', 'Close vs Open', 'Close vs Open %']].\
        style.set_caption("Largest Daily Fall by %").format(precision=2))

# Largest overnight fall by value
lof_amt = fmt_data[fmt_data['Open vs Prev Close'] == fmt_data['Open vs Prev Close'].min()]
display(lof_amt[['Open', 'Prev Close', 'Open vs Prev Close', 'Open vs Prev Close %']].\
        style.set_caption("Largest Overnight Fall by Value").format(precision=2))

# Largest overnight fall by %
lof_amt_pct = fmt_data[fmt_data['Open vs Prev Close %'] == fmt_data['Open vs Prev Close %'].min()]
display(lof_amt[['Open', 'Prev Close', 'Open vs Prev Close', 'Open vs Prev Close %']].\
        style.set_caption("Largest Overnight Fall by %").format(precision=2))

# Largest weekly fall by value
lwf_amt = fmt_data[fmt_data['Open vs Wk Ago Close'] == fmt_data['Open vs Wk Ago Close'].min()]
display(lwf_amt[['Open', 'Wk Ago Close', 'Open vs Wk Ago Close', 'Open vs Wk Ago Close %']].\
        style.set_caption("Largest Weekly Fall by Value").format(precision=2))

# Largest weekly fall by %
lwf_amt_pct = fmt_data[fmt_data['Open vs Wk Ago Close %'] == fmt_data['Open vs Wk Ago Close %'].min()]
display(lwf_amt[['Open', 'Wk Ago Close', 'Open vs Wk Ago Close', 'Open vs Wk Ago Close %']].\
        style.set_caption("Largest Weekly Fall by %").format(precision=2))

# Largest intra day swing for a bearish day. First filter for all bearish days (where close < open), then get largest swing as amt and pct
bear_days = fmt_data[fmt_data['Close'] < fmt_data['Open']]
lds_amt = bear_days[bear_days['Swing'] == bear_days['Swing'].max()]
display(lds_amt[['Open', 'Close', 'Swing', 'Swing %']].\
        style.set_caption("Largest Intraday Bear Swing by Amt").format(precision=2))

lds_pct = bear_days[bear_days['Swing %'] == bear_days['Swing %'].max()]
display(lds_pct[['Open', 'Close', 'Swing', 'Swing %']].\
        style.set_caption("Largest Intraday Bear Swing by %").format(precision=2))



,Open,Close,Close vs Open,Close vs Open %
Date,,,,
09-Nov-2021,1173.60,1023.50,-150.10,-12.79


,Open,Close,Close vs Open,Close vs Open %
Date,,,,
09-Nov-2021,1173.60,1023.50,-150.10,-12.79


,Open,Prev Close,Open vs Prev Close,Open vs Prev Close %
Date,,,,
08-Nov-2021,1149.79,1222.09,-72.30,-5.92


,Open,Prev Close,Open vs Prev Close,Open vs Prev Close %
Date,,,,
08-Nov-2021,1149.79,1222.09,-72.30,-5.92


,Open,Wk Ago Close,Open vs Wk Ago Close,Open vs Wk Ago Close %
Date,,,,
16-Nov-2021,1003.31,1222.09,-218.78,-17.90


,Open,Wk Ago Close,Open vs Wk Ago Close,Open vs Wk Ago Close %
Date,,,,
16-Nov-2021,1003.31,1222.09,-218.78,-17.90


,Open,Close,Swing,Swing %
Date,,,,
09-Nov-2021,1173.60,1023.50,162.98,16.11


,Open,Close,Swing,Swing %
Date,,,,
13-Jul-2020,331.80,299.41,64.78,22.02


## Now for the strategy
Calculate how much we need to raise from the stock sale to purchase the car, considering tax implications. This will be the stop loss posiotion. Calculate the target price before applying the stop loss taking into account the largest weekly fall.

In [6]:
# Target share price to purchase car. Car price is GBP 123000
# I hold 67 shares in a tax protected ISA and 52 in a standard account.
# The 52 shares are subject to 20% capital gains tax when they sell.

# Capital gains tax as % against total shares
cgt_pct = 52/(67+52)*20

# Amount I need to raise is car price + cgt %
car_price = 123000
tgt_amt = car_price + (car_price * cgt_pct / 100)

# Target share price in GBP is target amount / number shares held. Need to get in USD. Use 1.32 as FX rate
tgt_sh_pr_gbp = tgt_amt / (67+52)
tgt_sh_pr_usd = tgt_sh_pr_gbp * 1.32

# Target share price to allow for largest weekly fall before hitting stop loss. Largest weekly fall is 17.90%
tgt_rise_pr_usd = tgt_sh_pr_usd / ((100 - 17.9) / 100)

# Output
print(f"Stop loss of ${tgt_sh_pr_usd:,.2f} should be placed once TSLA reaches ${tgt_rise_pr_usd:,.2f} USD.\n" + \
      f"This will allow for historic maximum weekly drop to occour, whilst still raising £{tgt_amt:,.2f} to purchase car.\n" + \
      f"Amount to purchase car includes 20% CGT applied to shares outside of ISA.") 

Stop loss of $1,483.61 should be placed once TSLA reaches $1,807.08 USD.
This will allow for historic maximum weekly drop to occour, whilst still raising £133,749.58 to purchase car.
Amount to purchase car includes 20% CGT applied to shares outside of ISA.
